# Binary Knapsack Problem

A knapsack of limited capacity and a set of items are given. Each item is characterized by a weight and a profit. 
The goal is to select a subset of items such that the total profit is maximized and the sum of the weights does not exceed the capacity of the knapsack.

## Data

$I:$ set of items  
$q:$ capacity of the knapsack  
$w_i:$ weight of item $i \in I$  
$p_i:$ profit of item $i \in I$  

## Variables

$x_i:$ is $1$ if item $i\in I$ is selected in the knapsack

## Objective function

Maximize the profit
$$z^* = \max \sum_{i \in I} p_i \cdot x_i$$

## Constraints

Do not exceed the knapsack capacity
$$\sum_{i \in I} w_i \cdot x_i \le q$$

## Implementation

We first import Pyomo environment and create an abstract model:

In [ ]:
import pyomo.environ as pyopt

model = pyopt.AbstractModel("Knapsack problem")

We first declare the set of items: the initialization can be done by passing either a list or a function returning a list. In both cases the initialization does not occur until we recur to method `create_instance()`.

In [ ]:
model.I = pyopt.Set(initialize=list(range(1,11)))

We create parameters with random values. The initialization of parameters `p` and `w` is done using functions that will be called after the initialization of set `I`.

In [ ]:
import random

random.seed(0)

def getProfits(model):
    return {i: random.randrange(10, 50)  for i in model.I}

def getWeights(model):
    return {i: random.randrange(1, 10) for i in model.I}

model.p = pyopt.Param(model.I, initialize=getProfits)
model.w = pyopt.Param(model.I, initialize=getWeights)
model.q = pyopt.Param(initialize=20, mutable=True)

The we declare the set of variable `x`. Variables are binary and this implicitly impose that each item can be selected only once in the knapsack.

In [ ]:
model.x = pyopt.Var(model.I, within=pyopt.Binary)

The objective function maximize the sum of the profits of the selected items:

In [ ]:
def obj_function(model):
    return pyopt.sum(model.p[i] * model.x[i] for i in model.I)

model.z = pyopt.Objective(rule=obj_function, sense=pyopt.maximize)

Finally, we create the contraint ensuring that the total weight of the selected items does not exceeds the knapsack capacity. Constraints can be created using a function returning an expression

In [ ]:
def cons_capacity(model):
    return pyopt.sum( model.w[i] * model.x[i] for i in model.I) <= model.q

model.cons_capacity = pyopt.Constraint(rule=cons_capacity)

A summary of the abstract model can be displayed. All sets has size equal to 0 because the model has not been initialized yet.

In [ ]:
model.pprint()

To solve the problem we have to 
  1) initialize the instance by invoking method `create_instance()`
  2) create the interface to a solver
  3) ask the solver to start the optimization process

In [ ]:
instance = model.create_instance()
solver = pyopt.SolverFactory('glpk')
results = solver.solve(instance)

The result can be printed to display the status of the solver at the end of the optimization phase, the value of the objective function, and other useful information:

In [ ]:
print(results)

The instance is modified by setting the values of the variables. Details of the solution can be displayed invoking method `display()`

In [ ]:
instance.display()

Variable values can be easily retrieved after the problem is solved:

In [ ]:
print("x[1] = %d" % instance.x[1].value)
print("Selected items: %s" % ", ".join(str(i) for i, v in instance.x.iteritems() if v.value > 0)  )

When parameters are created setting argument `mutable=True`, they can be modified after creating the instance, and the modified instance can be solved again

In [ ]:
instance.q.set_value(30)
results = solver.solve(instance)

In such a way we can perform parametric analysis, for example to watch how the profit of the knapsack changes when the capacity increases:

In [ ]:
qs = list(range(20, 50, 5))

def parametric_analysis(q):
    instance.q.set_value(q)
    solver.solve(instance)
    return pyopt.value(instance.z)

import matplotlib.pyplot as plt
plt.plot(qs, list(map(parametric_analysis, qs)))
plt.ylabel('knapsack value')
plt.xlabel('knapsack capacity')
plt.show()